In [35]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import pandas as pd
import tkinter as tk
from tkinter import filedialog
import math
import scipy.integrate as spi
from datetime import datetime
from lets_plot import *

In [36]:
LetsPlot.setup_html()
# Initialize the lists
daysWILT = 66
temperature = [25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25]
realistic_temp = [15, 15.1022706, 15.170978, 15.2634094, 15.3968566, 15.512515, 15.6125191, 15.7318298, 15.8223274, 15.906782, 16.0168476, 16.1331547, 16.2749776, 16.4062527, 16.4660983, 16.5020481, 16.6565352, 16.7747617, 16.8227968, 16.8629698, 17.0229329, 17.158216, 17.3333158, 17.5015213, 17.5731757, 17.6980897, 17.8332817, 17.9366061, 18.1641707, 18.3629161, 18.6579604, 18.7586217, 18.8715818, 19.0127587, 19.3146276, 19.5048098, 19.6957149, 19.8611143, 20.0938469, 20.2731326, 20.330707, 20.4103492, 20.5826164, 20.7441389, 20.9280628, 21.0809132, 21.0894385, 21.2023533, 21.1669798, 21.4468847, 21.7182575, 21.966245, 22.1551721, 22.2590643, 22.3591437, 22.7121488, 23.262641, 23.5621818, 24.0056634, 24.4365625, 25.3755939, 25.6393261]
water_level = [150, 148.0201219, 146.1976759, 143.6619232, 140.6933556, 139.960227, 136.7467355, 135.2835277, 132.2718582, 131.0435903, 129.4067672, 129.0485111, 125.8347727, 122.8261396, 120.9875404, 117.6988996, 115.624174, 112.9174818, 109.8915115, 107.8650123, 105.8454984, 102.5158193, 100.28996, 98.4998298, 96.85433196, 93.6407309, 90.91916467, 88.73618655, 86.40312545, 84.0771963, 81.46610253, 79.54483091, 77.49875348, 74.52868738, 73.97248389, 71.44024876, 68.67037226, 66.94149999, 64.3862503, 62.210817, 59.72765212, 57.41940314, 55.75440326, 53.4578098, 50.58931065, 47.79474438, 45.5938958, 44.16095261, 42.12056342, 40.75708799, 39.13925171, 35.87322765, 33.20604052, 31.64167664, 29.48469869, 27.06309209, 25.16324147, 23.20904337, 20.54740315, 17.16194226, 15.35698605, 12.36700476, 9.525207548, 6.35970147, 4.727200498, 1.311416035]
N_inp = [0]
area = 10000 # meters
water_liters = [level * area * 1000 for level in water_level]


 # Parameters (based on literature

a = 0.9 #maximal nutrient uptake rate for the phytoplankton
c = 1 # maximal zooplankton ingestion rate
d = 0.1 # washout rate of the system
d1 = 0.1 # washout rate of the phytoplankton
d2 = 0.1 # washout rate of the zooplankton
λ = 0.45 # the rate at which saturation is achieved wtih increasing phytoplankton levels (per unit concentration)
γ = 0.15 #phytoplankton mortality rate
γ1 = 0.08 #nutrient recycle rate after the death of the phytoplankton
δ = 0.6 #the fraction of zooplankton nutrient conversion,
ε = 0.15   # Zooplankton death rate (range: 0.05-0.2, source: Hansen et al., 1997)
ε1 = 0.05  #nutrient recycle rate after the death of the zooplankton
k = 0.22 #Michaelis Menten constant.
x = 0.06 #predetation rate
# Temperature dependency parameters phytoplankton ctmi
def ctmi_model(T, Tmin, Topt, Tmax, µopt):
    nominator = (T - Tmax) * (T - Tmin)**2
    denominator = (Topt - Tmin) * ((Topt - Tmin) * (T - Topt) - (Topt - Tmax) * (Topt + Tmin - 2 * T))
    μ = µopt * nominator / denominator
    return μ

# Set the parameter values
Tmin = 5
Topt = 25
Tmax = 40
µopt = 1.35
# Initial conditions

P0 = 1.0 # Phytoplankton (µg C L^-1)
Z0 = 1.0  # Zooplankton concentration (µg C L^-1)
D0 = 4.0  # Detritus concentration (µg C L^-1)
N0 = 6.0

# Time parameters
dt = 0.1  # Time step
time_steps_per_day = 10
timesteps = int((daysWILT) / dt)
days = daysWILT
print(timesteps)

def interpolate_data(data, timesteps):
    interpolated_data = np.interp(
        np.linspace(0, len(data) - 1, timesteps),
        np.arange(len(data)),
        data
    )

    return interpolated_data.tolist()
expanded_water_level = interpolate_data(water_level, timesteps)
expanded_temperature = interpolate_data(realistic_temp, timesteps)
expanded_water_liters = interpolate_data(water_liters, timesteps)
N_inp_exp = interpolate_data(N_inp, timesteps)
# Initialize arrays

N_Nutrient = np.zeros(timesteps)
P_Nutrient = np.zeros(timesteps)
N = np.zeros(timesteps)
P = np.zeros(timesteps)
Z = np.zeros(timesteps)
D = np.zeros(timesteps)
P_prod = np.zeros(timesteps) # Phytoplankton production rate
Z_prod = np.zeros(timesteps) # Zooplankton production rate
P_loss = np.zeros(timesteps) # Phytoplankton loss rate
Z_loss = np.zeros(timesteps) # Zooplankton loss rate

#N_Nutrient[0] = N_Nutrient0
#P_Nutrient[0] = P_Nutrient0
N[0] = N0
P[0] = P0
Z[0] = Z0
D[0] = D0
# Euler method for numerical integration
for t in range(1, timesteps):    # Get the water temperature for this time step
    #t_w_c = temperature[t]
    #p_grow = phytoplankton.calculate_growthrate(t_w_c)
    T = expanded_temperature[t]
    p_grow = ctmi_model(T, Tmin, Topt, Tmax, µopt)
    # print(f"At temperature {T}°C, the growth rate is {p_grow}.")
    wp = 1 - math.exp(-λ * P[t - 1])
    un = N[t-1] / (k + N[t-1])
    dNdt = d * (N[0] - N[t-1]) - a * P[t-1] * un + (1 - δ) * c * Z[t-1] * wp + γ1 * P[t-1] + ε1 * Z[t-1] + N_inp_exp[t-1]
    dPdt = p_grow * P[t-1] * un - c * Z[t-1] * wp - (γ + d1) * P[t-1]
    dZdt = Z[t-1] * (δ * c * wp - (ε + x + d2))
    P[t] = ((P[t - 1] + dPdt * dt)*expanded_water_liters[t-1])/expanded_water_liters[t]
    Z[t] = ((Z[t - 1] + dZdt * dt)*expanded_water_liters[t-1])/expanded_water_liters[t]
    N[t] = ((N[t - 1] + dNdt * dt )*expanded_water_liters[t-1])/expanded_water_liters[t]
    D[t] = 12 - P[t] - Z[t] - N[t]
    P_prod[t] = p_grow * P[t - 1] * un
    Z_prod[t] = Z[t - 1] * (δ * c * wp)
    P_loss[t] = (γ + d1) * P[t - 1]
    Z_loss[t] = (ε + x + d2) * Z[t - 1]

660


In [37]:
w, h = 600, 400

def themeWithFlavor(plot, theme, title):
    bunch = GGBunch()
    bunch.add_plot(plot + theme + ggtitle(title), 0, 0, w, h)
    bunch.show()

In [38]:
# Extract N, P and Z arrays from the simulation
N = np.array(N)
P = np.array(P)
Z = np.array(Z)
T = np.array(expanded_temperature)
WL = np.array(expanded_water_level)
Nin = np.array(N_inp_exp)

x = np.linspace(0, days, timesteps)

# Create a dataframe
data = {'Time': np.arange(len(P)), 'N': N,'P': P, 'Z': Z, 'x': x}
df = pd.DataFrame({'Time': np.arange(len(P)), 'P': P, 'Z': Z, 'N': N, 'x': x})
df = pd.melt(df, id_vars=['x'], value_vars=['P', 'N', 'Z'])
Concentrationgraph = ggplot(df, aes(x='x', y='value', group='variable')) + \
     geom_line(aes(color='variable'), size=1, alpha=0.5) + \
     ggtitle('Concentration graph') + \
     ylim(0, 10) + \
     ylab("Concentration (µg L^-1)") + \
     xlab("Days")
#geom_line(stat='smooth', color='red', linetype='longdash')



themeWithFlavor(Concentrationgraph, theme_grey(), "Concentration NPZ over time")
#ggsave(Concentrationgraph, input('givename')+'plot.html', path='E:/Deltares/figures', iframe=False)

In [39]:
df = pd.DataFrame({'Time': np.arange(len(P)), 'Water Level': WL, 'Temperature': T, 'Nutrient input': Nin, 'Days': x})
WLp = ggplot({'Days': x, 'Water Level': WL}, aes(x='Days', y='Water Level')) + ggsize(w/2, h/3)
Tp = ggplot({'Days': x, 'Temperature': T}, aes(x='Days', y='Temperature')) + ggsize(w/2, h/3)
Ninp = ggplot({'Days': x, 'Nutrient input': Nin}, aes(x='Days', y='Nutrient input')) + ggsize(w/2, h/3)

bunch = GGBunch()
bunch.add_plot(WLp + geom_line() + theme_grey(), 1*w, 0)
bunch.add_plot(Tp + geom_line() +ylim(10, 40) + theme_grey(), 1*w, h/3)
bunch.add_plot(Ninp + geom_line() + theme_grey(), 1*w, h/3*2)
bunch.add_plot(Concentrationgraph + theme_grey(), 0, 0)

bunch.show()

# themeWithFlavor(bunch.show(), theme_grey(), "Production graph")
ggsave(bunch, input('givename')+'_scenarioplot.png', path='E:/Deltares/figures', iframe=False)

'E:\\Deltares\\figures\\Realistic_20230322-20230531_scenarioplot.png'

In [42]:
# Extract N, P and Z arrays from the simulation
P = np.array(P_prod)
Z = np.array(Z_prod)
x = np.linspace(0, days, timesteps)

# Create a dataframe
data = {'Time': np.arange(len(P)), 'P': P, 'Z': Z, 'x': x}
df = pd.DataFrame({'Time': np.arange(len(P)), 'P': P, 'Z': Z, 'x': x})
df = pd.melt(df, id_vars=['x'], value_vars=['P', 'Z'])
productiongraph = ggplot(df, aes(x='x', y='value', group='variable')) + \
    geom_line(aes(color='variable'), size=1, alpha=0.5) + \
    ggtitle('Production graph') + \
    ylim(0, 10) + \
    ylab("Production (µg L^-1 dt-1)") + \
    xlab("Days")
    #geom_line(stat='smooth', color='red', linetype='longdash')



themeWithFlavor(productiongraph, theme_grey(), "Production")
ggsave(productiongraph, input('givename')+'productionplot.png', path='E:/Deltares/figures', iframe=False)





'E:\\Deltares\\figures\\realisticproductionproductionplot.png'

In [43]:
# Extract N, P and Z arrays from the simulation
P = np.array(P_loss)
Z = np.array(Z_loss)
x = np.linspace(0, days, timesteps)

# Create a dataframe
data = {'Time': np.arange(len(P)), 'P': P, 'Z': Z, 'x': x}
df = pd.DataFrame({'Time': np.arange(len(P)), 'P': P, 'Z': Z, 'x': x})
df = pd.melt(df, id_vars=['x'], value_vars=['P', 'Z'])
lossesgraph = ggplot(df, aes(x='x', y='value', group='variable')) + \
     geom_line(aes(color='variable'), size=1, alpha=0.5) + \
     ggtitle('Losses graph') + \
     ylim(0, 10) + \
     ylab("Losses (µg L^-1 dt-1)") + \
     xlab("Days")
#geom_line(stat='smooth', color='red', linetype='longdash')



themeWithFlavor(lossesgraph, theme_grey(), "Losses")
ggsave(lossesgraph, input('givename')+'lossesplot.png', path='E:/Deltares/figures', iframe=False)

'E:\\Deltares\\figures\\realisticlossesplot.png'